https://towardsdatascience.com/regression-models-with-multiple-target-variables-8baa75aacd

http://www.cs.put.poznan.pl/kdembczynski/pdf/multi-target_prediction.pdf

https://github.com/Microsoft/LightGBM/issues/524

https://stackoverflow.com/questions/39540123/muti-output-regression-in-xgboost

http://hamelg.blogspot.ru/2015/11/python-for-data-analysis-part-17.html

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None
from sklearn.preprocessing import Imputer

In [2]:
df_train = pd.read_csv('data/raiffeisen/train_set.csv', low_memory=False)

In [3]:
df_test = pd.read_csv('data/raiffeisen/test_set.csv', low_memory=False)

In [4]:
df_test.customer_id.nunique()

9997

In [5]:
df_train['transaction_date'] = pd.to_datetime(df_train['transaction_date'])
df_test['transaction_date'] = pd.to_datetime(df_test['transaction_date'])

In [5]:
df_train.head()

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_adress_lat,pos_adress_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
0,2.884034,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,11606fde0c814ce78e0d726e39a0a5ee,2017-07-15,59.847,30.177
1,2.775633,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,e9647a5e1eacfb06713b6af755ccc595,2017-10-27,59.847,30.177
2,3.708368,NaN,NaN,NaN,St Petersburg,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5992,"PR.MARSHALA ZHUKOVA,31St Petersburg190000 7...",59.858198,30.229024,df06c1fcd3718a514535ae822785f716,2017-10-03,59.847,30.177
3,2.787498,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,6c5e5793ebc984fb72875feffff62854,2017-09-09,59.847,30.177
4,2.892510,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,0576445d74e374c92c0902e612fca356,2017-07-06,59.847,30.177


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224734 entries, 0 to 1224733
Data columns (total 18 columns):
amount              1224734 non-null float64
atm_address         269473 non-null object
atm_address_lat     184680 non-null float64
atm_address_lon     184680 non-null float64
city                1224725 non-null object
country             1224734 non-null object
currency            1224690 non-null float64
customer_id         1224734 non-null object
home_add_lat        1224718 non-null float64
home_add_lon        1224718 non-null float64
mcc                 1224734 non-null int64
pos_address         924991 non-null object
pos_adress_lat      942614 non-null float64
pos_adress_lon      942614 non-null float64
terminal_id         1224705 non-null object
transaction_date    1224690 non-null datetime64[ns]
work_add_lat        664660 non-null float64
work_add_lon        664660 non-null float64
dtypes: datetime64[ns](1), float64(10), int64(1), object(6)
memory usage: 168.2+ MB


In [6]:
column_1 = df_train['transaction_date']
df_train['year'] = column_1.dt.year
df_train['month'] = column_1.dt.month
df_train['day'] = column_1.dt.day
df_train['hour'] = column_1.dt.hour
df_train['dayofyear'] = column_1.dt.dayofyear
df_train['week'] = column_1.dt.week
df_train['weekofyear'] = column_1.dt.weekofyear
df_train['dayofweek'] = column_1.dt.dayofweek
df_train['weekday'] = column_1.dt.weekday
df_train['quarter'] = column_1.dt.quarter

In [7]:
column_1 = df_test['transaction_date']
df_test['year'] = column_1.dt.year
df_test['month'] = column_1.dt.month
df_test['day'] = column_1.dt.day
df_test['hour'] = column_1.dt.hour
df_test['dayofyear'] = column_1.dt.dayofyear
df_test['week'] = column_1.dt.week
df_test['weekofyear'] = column_1.dt.weekofyear
df_test['dayofweek'] = column_1.dt.dayofweek
df_test['weekday'] = column_1.dt.weekday
df_test['quarter'] = column_1.dt.quarter

### Random forest regressor with time series

In [8]:
X_train = df_train.drop(['atm_address', 'city', 'country', 'customer_id', 'pos_address', 'terminal_id', 'home_add_lat', 'home_add_lon', 'work_add_lat', 'work_add_lon', 'transaction_date'], axis=1)

In [9]:
X_train['atm_address_lat'] = X_train['atm_address_lat'].fillna(X_train['pos_adress_lat'])
X_train['atm_address_lon'] = X_train['atm_address_lat'].fillna(X_train['pos_adress_lon'])

In [10]:
X_train['currency'] = X_train['currency'].fillna(0)

In [11]:
X_train['pos_adress_lat'] = X_train['pos_adress_lat'].fillna(X_train['atm_address_lat'])
X_train['pos_adress_lon'] = X_train['pos_adress_lon'].fillna(X_train['atm_address_lon'])

In [66]:
X_train.head()

,amount,atm_address_lat,atm_address_lon,currency,mcc,pos_adress_lat,pos_adress_lon,year,month,day,hour,dayofyear,week,weekofyear,dayofweek,weekday,quarter
0,2.884034,59.844072,59.844072,643.0,5261,59.844072,30.179153,2017.0,7.0,15.0,0.0,196.0,28.0,28.0,5.0,5.0,3.0
1,2.775633,59.844072,59.844072,643.0,5261,59.844072,30.179153,2017.0,10.0,27.0,0.0,300.0,43.0,43.0,4.0,4.0,4.0
2,3.708368,59.858198,59.858198,643.0,5992,59.858198,30.229024,2017.0,10.0,3.0,0.0,276.0,40.0,40.0,1.0,1.0,4.0
3,2.787498,59.844072,59.844072,643.0,5261,59.844072,30.179153,2017.0,9.0,9.0,0.0,252.0,36.0,36.0,5.0,5.0,3.0
4,2.892510,59.844072,59.844072,643.0,5261,59.844072,30.179153,2017.0,7.0,6.0,0.0,187.0,27.0,27.0,3.0,3.0,3.0


In [67]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224734 entries, 0 to 1224733
Data columns (total 17 columns):
amount             1224734 non-null float64
atm_address_lat    1127294 non-null float64
atm_address_lon    1127294 non-null float64
currency           1224734 non-null float64
mcc                1224734 non-null int64
pos_adress_lat     1127294 non-null float64
pos_adress_lon     1127294 non-null float64
year               1224690 non-null float64
month              1224690 non-null float64
day                1224690 non-null float64
hour               1224690 non-null float64
dayofyear          1224690 non-null float64
week               1224690 non-null float64
weekofyear         1224690 non-null float64
dayofweek          1224690 non-null float64
weekday            1224690 non-null float64
quarter            1224690 non-null float64
dtypes: float64(16), int64(1)
memory usage: 158.8 MB


In [12]:
y_train = df_train[['home_add_lat', 'home_add_lon', 'work_add_lat', 'work_add_lon']]

In [69]:
y_train.head()

,home_add_lat,home_add_lon,work_add_lat,work_add_lon
0,59.851,30.232,59.847,30.177
1,59.851,30.232,59.847,30.177
2,59.851,30.232,59.847,30.177
3,59.851,30.232,59.847,30.177
4,59.851,30.232,59.847,30.177


In [70]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224734 entries, 0 to 1224733
Data columns (total 4 columns):
home_add_lat    1224718 non-null float64
home_add_lon    1224718 non-null float64
work_add_lat    664660 non-null float64
work_add_lon    664660 non-null float64
dtypes: float64(4)
memory usage: 37.4 MB


In [13]:
X_train = X_train.fillna(X_train.mean())

In [72]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224734 entries, 0 to 1224733
Data columns (total 17 columns):
amount             1224734 non-null float64
atm_address_lat    1224734 non-null float64
atm_address_lon    1224734 non-null float64
currency           1224734 non-null float64
mcc                1224734 non-null int64
pos_adress_lat     1224734 non-null float64
pos_adress_lon     1224734 non-null float64
year               1224734 non-null float64
month              1224734 non-null float64
day                1224734 non-null float64
hour               1224734 non-null float64
dayofyear          1224734 non-null float64
week               1224734 non-null float64
weekofyear         1224734 non-null float64
dayofweek          1224734 non-null float64
weekday            1224734 non-null float64
quarter            1224734 non-null float64
dtypes: float64(16), int64(1)
memory usage: 158.8 MB


In [14]:
y_train = y_train.fillna(y_train.mean())

In [74]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224734 entries, 0 to 1224733
Data columns (total 4 columns):
home_add_lat    1224734 non-null float64
home_add_lon    1224734 non-null float64
work_add_lat    1224734 non-null float64
work_add_lon    1224734 non-null float64
dtypes: float64(4)
memory usage: 37.4 MB


In [17]:
%%time
regr = RandomForestRegressor(max_depth=30, random_state=0)

Wall time: 0 ns


In [18]:
regr_multirf = MultiOutputRegressor(regr)

In [20]:
regr_multirf.fit(X_train, y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False),
           n_jobs=1)

In [21]:
df_test.head()

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,year,month,day,hour,dayofyear,week,weekofyear,dayofweek,weekday,quarter
0,2.211818,NaN,NaN,NaN,IVANTEEVKA,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,"5,411",2-1 TOLMACHEVA STRIVANTEEVKA141280 RUSRUS,55.967487,37.913682,ff0476dae4b098a7b16aabe93d4268df,2017-08-24,2017.0,8.0,24.0,0.0,236.0,34.0,34.0,3.0,3.0,3.0
1,1.331379,NaN,NaN,NaN,IVANTEEVKA,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,"4,111","SOVETSKIJ, 32IVANTEEVKA141282 RUSRUS",55.971296,37.905184,7cfd9a60282459d4692ecc85b856072e,2017-08-12,2017.0,8.0,12.0,0.0,224.0,32.0,32.0,5.0,5.0,3.0
2,2.608004,NaN,NaN,NaN,PUSHKINO,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,"5,411","105,KRASOARMEYSKOE SHPUSHKINO141206 RUSRUS",56.016591,37.909098,7e5a532f0029861d8a9c4f0479b9450b,2017-06-17,2017.0,6.0,17.0,0.0,168.0,24.0,24.0,5.0,5.0,2.0
3,1.916752,NaN,NaN,NaN,IVANTEEVKA,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,"5,411","G. IVANTEEVKA, UL.TOLMACHEVA, D.6IVANTEEVKA141...",55.964507,37.937912,2afe7d1bc61b86c449f413bdf2119032,2017-08-12,2017.0,8.0,12.0,0.0,224.0,32.0,32.0,5.0,5.0,3.0
4,1.981067,NaN,NaN,NaN,MOSCOW,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,"5,814",5 KOMSOMOLSKAYA SQMOSCOW101000 RUSRUS,55.776802,37.657352,ab4f00601ff1d949afc59ee3f804c79c,2017-04-26,2017.0,4.0,26.0,0.0,116.0,17.0,17.0,2.0,2.0,2.0


In [22]:
df_test.shape

(1265470, 24)

In [35]:
X_test = df_test.drop(['atm_address', 'city', 'country', 'customer_id', 'pos_address', 'terminal_id', 'transaction_date'], axis=1)

In [36]:
X_test['mcc'] = pd.to_numeric(X_test['mcc'].str.replace(',',''), errors='coerce')

In [37]:
X_test.head()

,amount,atm_address_lat,atm_address_lon,currency,mcc,pos_address_lat,pos_address_lon,year,month,day,hour,dayofyear,week,weekofyear,dayofweek,weekday,quarter
0,2.211818,NaN,NaN,643.0,5411,55.967487,37.913682,2017.0,8.0,24.0,0.0,236.0,34.0,34.0,3.0,3.0,3.0
1,1.331379,NaN,NaN,643.0,4111,55.971296,37.905184,2017.0,8.0,12.0,0.0,224.0,32.0,32.0,5.0,5.0,3.0
2,2.608004,NaN,NaN,643.0,5411,56.016591,37.909098,2017.0,6.0,17.0,0.0,168.0,24.0,24.0,5.0,5.0,2.0
3,1.916752,NaN,NaN,643.0,5411,55.964507,37.937912,2017.0,8.0,12.0,0.0,224.0,32.0,32.0,5.0,5.0,3.0
4,1.981067,NaN,NaN,643.0,5814,55.776802,37.657352,2017.0,4.0,26.0,0.0,116.0,17.0,17.0,2.0,2.0,2.0


In [38]:
X_test['atm_address_lat'] = X_test['atm_address_lat'].fillna(X_test['pos_address_lat'])
X_test['atm_address_lon'] = X_test['atm_address_lat'].fillna(X_test['pos_address_lon'])

In [39]:
X_test['currency'] = X_test['currency'].fillna(0)

In [40]:
X_test['pos_address_lat'] = X_test['pos_address_lat'].fillna(X_test['atm_address_lat'])
X_test['pos_address_lon'] = X_test['pos_address_lon'].fillna(X_test['atm_address_lon'])

In [41]:
X_test = X_test.fillna(X_test.mean())

In [42]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1265470 entries, 0 to 1265469
Data columns (total 17 columns):
amount             1265470 non-null float64
atm_address_lat    1265470 non-null float64
atm_address_lon    1265470 non-null float64
currency           1265470 non-null float64
mcc                1265470 non-null int64
pos_address_lat    1265470 non-null float64
pos_address_lon    1265470 non-null float64
year               1265470 non-null float64
month              1265470 non-null float64
day                1265470 non-null float64
hour               1265470 non-null float64
dayofyear          1265470 non-null float64
week               1265470 non-null float64
weekofyear         1265470 non-null float64
dayofweek          1265470 non-null float64
weekday            1265470 non-null float64
quarter            1265470 non-null float64
dtypes: float64(16), int64(1)
memory usage: 164.1 MB


In [43]:
y_multirf = regr_multirf.predict(X_test)

In [44]:
df_trees = pd.DataFrame(y_multirf)

In [45]:
df_trees = df_trees.round(5)

In [46]:
df_trees.head()

,0,1,2,3
0,55.88447,37.83348,56.07442,39.25557
1,55.75552,37.78137,56.00836,37.67931
2,55.90234,37.75037,55.98961,39.05289
3,55.92366,37.77062,55.93086,39.18375
4,55.57842,38.12292,56.11052,39.43046


In [47]:
result = pd.concat([df_test['customer_id'], df_trees], axis=1)

In [48]:
result.head()

,customer_id,0,1,2,3
0,00fd410f5c580c8351cafa88d82b60f3,55.88447,37.83348,56.07442,39.25557
1,00fd410f5c580c8351cafa88d82b60f3,55.75552,37.78137,56.00836,37.67931
2,00fd410f5c580c8351cafa88d82b60f3,55.90234,37.75037,55.98961,39.05289
3,00fd410f5c580c8351cafa88d82b60f3,55.92366,37.77062,55.93086,39.18375
4,00fd410f5c580c8351cafa88d82b60f3,55.57842,38.12292,56.11052,39.43046


In [49]:
result_mean = result.groupby(['customer_id']).mean()

In [50]:
result_median = result.groupby(['customer_id']).median()

In [56]:
result_mean.head()

,0,1,2,3
customer_id,,,,
00021683ccb416637fe9a4cd35e4606e,55.018648,81.812454,55.596530,68.454446
0002d0f8a642272b41c292c12ab6e602,52.610758,43.054356,55.014482,40.522652
0004d182d9fede3ba2534b2d5e5ad27e,47.816585,40.547469,50.726676,40.375395
0008c2445518c9392cb356c5c3db3392,52.119316,44.622993,54.107614,41.598741
000b373cc4969c0be8e0933c08da67e1,56.093814,43.331304,56.215500,41.680688


In [51]:
result_mean.shape

(9997, 4)

In [57]:
result_mean.to_csv('raif_mean.csv', index=True)

In [58]:
result_median.to_csv('raif_median.csv', index=True)